In [31]:
import math
import numpy as np 
import pandas as pd
import os
import time

In [32]:
path = r'%s' % os.getcwd().replace('\\','/')
path

'C:/Users/138035/Documents/Projects/Core Connect/CoreConnectML'

In [33]:
start = time.time()

# Directory where the data (csvs) are, be sure to change to match your personal directory
dir_ = path + '/Data/'

print(dir_)

# Global counter to sort through stock data files
file_counter_ = 1

# Change the working directory to where the data sits
os.chdir(dir_)

C:/Users/138035/Documents/Projects/Core Connect/CoreConnectML/Data/


In [34]:
# Function to calculate the True Range in each row
def calc_tr_(row):

    high_ = row['High']
    low_ = row['Low']
    open_ = row['Open']
    close_ = row['Close']
    volume_ = row['Volume']
    prev_close_ = row['Prev Close']

    max_1_ = high_ - low_
    max_2_ = abs(high_ - prev_close_)
    max_3_ = abs(low_- prev_close_)

    TR_ = max(max_1_,max_2_,max_3_)

    return TR_

In [35]:
# Loop through the directory to ingest each file
for file_ in os.listdir(dir_):
    # Read the csv
    df_stocks_temp_ = pd.read_csv(file_)
    # Grab the ticker
    df_stocks_temp_['Ticker'] = file_.split(".")[0]
    # Populate the previous close
    df_stocks_temp_['Prev Close'] = df_stocks_temp_['Close'].shift(1)
    # Calculate the True Range for each row for the ATR calculation
    df_stocks_temp_['TR'] = df_stocks_temp_.apply(calc_tr_,axis=1)
    # Create increment for ATR calculation
    atr_counter_ = 1
    # Find the length of the TR column then find how many iterations of 14 days 
    length_atr_ = len(df_stocks_temp_['TR'])
    iterations_ = length_atr_%14

    # For loop to grab frames of TR data
    for i in range(0,iterations_):
        # Grab TR data in increments of 14
        df_temp_atr_ = df_stocks_temp_['TR'][(atr_counter_-1)*14:atr_counter_*14]
        # Length of values to iterate and calculate over
        length_df_temp_atr_ = len(df_temp_atr_)
        # Loop to Calculate ATR
        for j in range(0,length_df_temp_atr_):
            # Grab the True Range value
            ATR_temp_ = df_temp_atr_[((atr_counter_-1)*14)+j]
            # If the value is being initializated then populate
            if i == 0:
                ATR_ = ATR_temp_
            # If past initialization then add the TR values for the iterable period
            else:
                ATR_ = ATR_ + ATR_temp_
        # Populate the master data frame with the ATR values for the respective stock 
        df_stocks_temp_['ATR'] = ATR_
        # Increment ATR Counter 
        atr_counter_ += 1

    # Drop TR Column here

    
    # If the master data frame is being initialized then populate
    if file_counter_ == 1:
        df_stocks_ = df_stocks_temp_
    # Else append dataframe
    else:
        df_stocks_ = df_stocks_.append(df_stocks_temp_)
    # Clear temp dataframe
    df_stocks_temp_ = df_stocks_temp_.iloc[0:0]

    # Increment file count
    file_counter_ += 1

In [36]:
# Reset the dataframe index
df_stocks_ = df_stocks_.set_index(['Date','Ticker'])
# Sort dataframe by date
df_stocks_ = df_stocks_.sort_values('Date')

In [37]:
end = time.time()
print("Execution time:", end - start)

Execution time: 6.321531057357788


In [38]:
import psutil
# gives a single float value
psutil.cpu_percent()
# gives an object with many fields
psutil.virtual_memory()
# you can convert that object to a dictionary 
dict(psutil.virtual_memory()._asdict())

{'total': 21344120832,
 'available': 9578168320,
 'percent': 55.1,
 'used': 11765952512,
 'free': 9578168320}